In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
from tqdm import tqdm  # 진행률 표시
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 경고 메시지 무시
warnings.filterwarnings('ignore')

# 데이터가 있는 폴더 경로 설정 (상대 경로)
DATA_DIR = './open_track1'

# 주요 파일 경로 정의
TRAIN_PATH = os.path.join(DATA_DIR, 'train.csv')
TEST_META_PATH = os.path.join(DATA_DIR, 'test.csv')
SUBMISSION_PATH = os.path.join(DATA_DIR, 'sample_submission.csv')

# 경로 확인
print(f"데이터 경로 설정 완료: {DATA_DIR}")
if not os.path.exists(TRAIN_PATH):
    print("경고: train.csv 파일을 찾을 수 없습니다. 현재 폴더 위치를 확인해주세요.")
else:
    print("파일 경로 확인 완료!")

📂 데이터 경로 설정 완료: ./open_track1
✅ 파일 경로 확인 완료!


In [ ]:
# 데이터 로드
print("메타데이터 로딩 중...")
train_df = pd.read_csv(TRAIN_PATH)
test_meta_df = pd.read_csv(TEST_META_PATH)

print(f"학습 데이터(Train) 크기: {train_df.shape}")
print(f"테스트 메타데이터(Test Meta) 크기: {test_meta_df.shape}")

# 데이터 확인 (상위 3개 행)
display(train_df.head(3))

⏳ 메타데이터 로딩 중...
✅ 학습 데이터(Train) 크기: (356721, 15)
✅ 테스트 메타데이터(Test Meta) 크기: (2414, 3)


,game_id,period_id,episode_id,time_seconds,team_id,player_id,action_id,type_name,result_name,start_x,start_y,end_x,end_y,is_home,game_episode
0,126283,1,1,0.667,2354,344559,0,Pass,Successful,52.418205,33.485444,31.322445,38.274752,True,126283_1
1,126283,1,1,3.667,2354,250036,2,Pass,Successful,32.013240,38.100808,37.371285,30.632980,True,126283_1
2,126283,1,1,4.968,2354,500145,4,Carry,NaN,37.371285,30.632980,38.391570,24.613144,True,126283_1


In [4]:
# 1. 데이터 정렬 (게임 > 에피소드 > 액션 순서)
train_df = train_df.sort_values(['game_id', 'episode_id', 'action_id'])

# 2. 각 에피소드의 '마지막 행'만 추출 (우리가 예측해야 할 시점)
train_last_actions = train_df.drop_duplicates(subset=['game_episode'], keep='last').copy()

# 3. 학습에 사용할 피처(Feature)와 타겟(Target) 정의
# 피처: 시작 좌표, 행동 종류, 팀 ID, 시간
features = ['start_x', 'start_y', 'type_name', 'team_id', 'time_seconds']
# 타겟: 종료 좌표 (예측 대상)
target_cols = ['end_x', 'end_y']

# 4. X(입력), y(정답) 데이터 분리
X_train_raw = train_last_actions[features]
y_train = train_last_actions[target_cols]

print(f"전처리 완료된 학습 데이터 크기: {X_train_raw.shape}")
display(X_train_raw.head())

전처리 완료된 학습 데이터 크기: (15435, 5)


,start_x,start_y,type_name,team_id,time_seconds
48,101.054205,52.757324,Pass,2354,124.367
51,100.221660,0.066028,Pass,2354,194.067
58,5.457847,66.343459,Pass,4639,215.033
61,60.171825,2.108816,Pass,2354,227.733
84,33.185670,26.107444,Pass,2354,255.767


In [5]:
test_data_list = []

# test.csv의 파일 경로를 하나씩 순회하며 읽기
for index, row in tqdm(test_meta_df.iterrows(), total=len(test_meta_df)):
    # 1. 경로 수정: ./test/... -> ./open_track1/test/...
    original_path = row['path']
    clean_path = original_path.lstrip('./').lstrip('.\\') # 앞쪽 ./ 제거
    file_path = os.path.join(DATA_DIR, clean_path) # 올바른 경로 결합
    
    episode_id = row['game_episode']
    
    try:
        # 개별 CSV 로드
        df = pd.read_csv(file_path)
        
        # 마지막 행 추출 (이 데이터로 다음 좌표를 예측해야 함)
        last_row = df.iloc[-1]
        
        # 리스트에 추가
        test_data_list.append({
            'game_episode': episode_id,
            'start_x': last_row['start_x'],
            'start_y': last_row['start_y'],
            'type_name': last_row['type_name'],
            'team_id': last_row['team_id'],
            'time_seconds': last_row['time_seconds']
        })
        
    except FileNotFoundError:
        print(f"파일을 찾을 수 없음: {file_path}")
    except Exception as e:
        print(f"읽기 오류 발생 {file_path}: {e}")

# 리스트를 DataFrame으로 변환
X_test_raw = pd.DataFrame(test_data_list)
print(f"생성된 테스트 데이터 크기: {X_test_raw.shape}")
display(X_test_raw.head())

100%|██████████| 2414/2414 [00:45<00:00, 53.04it/s]

생성된 테스트 데이터 크기: (2414, 6)


,game_episode,start_x,start_y,type_name,team_id,time_seconds
0,153363_1,57.801450,9.203664,Pass,4639,83.133
1,153363_2,19.552575,47.239464,Pass,4220,114.967
2,153363_6,20.025705,63.088870,Pass,4220,261.289
3,153363_7,38.639790,8.072620,Pass,4639,295.167
4,153363_8,66.560550,15.260288,Pass,4220,348.149


In [6]:
le = LabelEncoder()

# Train과 Test의 모든 이벤트 타입(type_name)을 통합하여 학습
# (Test에만 있는 새로운 행동이 있을 수도 있으므로)
all_types = pd.concat([X_train_raw['type_name'], X_test_raw['type_name']]).unique()
le.fit(all_types)

# 원본 데이터 복사 후 변환 적용
X_train = X_train_raw.copy()
X_test = X_test_raw.copy()

X_train['type_name'] = le.transform(X_train_raw['type_name'])
X_test['type_name'] = le.transform(X_test_raw['type_name'])

# 테스트 데이터에서 식별자(ID) 분리 (모델에는 입력하지 않음)
# game_episode 컬럼이 있다면 분리해둠
if 'game_episode' in X_test.columns:
    test_ids = X_test['game_episode']
    X_test = X_test.drop(columns=['game_episode'])
else:
    test_ids = test_meta_df['game_episode']

display(X_train.head(3))

,start_x,start_y,type_name,team_id,time_seconds
48,101.054205,52.757324,0,2354,124.367
51,100.221660,0.066028,0,2354,194.067
58,5.457847,66.343459,0,4639,215.033


In [7]:
# 1. 검증을 위해 Train 데이터를 8:2로 분할
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# 2. 모델 정의
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

# 3. 학습 (검증용 데이터로)
model.fit(X_tr, y_tr)

# 4. 검증 및 점수 확인
val_preds = model.predict(X_val)
rmse_x = np.sqrt(mean_squared_error(y_val['end_x'], val_preds[:, 0]))
rmse_y = np.sqrt(mean_squared_error(y_val['end_y'], val_preds[:, 1]))
mean_rmse = (rmse_x + rmse_y) / 2

print(f"[검증 결과] RMSE - X축: {rmse_x:.4f}, Y축: {rmse_y:.4f}")
print(f"평균 RMSE: {mean_rmse:.4f}")

# 5. 전체 데이터로 최종 학습 (제출 성능 극대화)
print("전체 데이터로 재학습 중...")
model.fit(X_train, y_train)
print("최종 모델 학습 완료")

[검증 결과] RMSE - X축: 13.9016, Y축: 14.9452
평균 RMSE: 14.4234
전체 데이터로 재학습 중...
최종 모델 학습 완료
